# TCN Classification with MR Dataset
<hr>

We will build a text classification model using TCN model on the Movie Review Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MR/MR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10662, 3)


,sentence,label,split
0,"simplistic , silly and tedious .",0,train
1,"it 's so laddish and juvenile , only teenage b...",0,train
2,exploitative and largely devoid of the depth o...,0,train
3,garbus discards the potential for pathological...,0,train
4,a visually flashy but narratively opaque and e...,0,train
...,...,...,...
10657,both exuberantly romantic and serenely melanch...,1,train
10658,mazel tov to a film about a family 's joyous l...,1,train
10659,standing in the shadows of motown is the best ...,1,train
10660,it 's nice to see piscopo again after all thes...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10662 non-null  object
 1   label     10662 non-null  int32 
 2   split     10662 non-null  object
dtypes: int32(1), object(2)
memory usage: 208.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5331,5331
1,5331,5331


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'simplistic , silly and tedious .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
Into a sequence of int: [3, 544, 1838, 13, 3909, 3366, 4, 658, 2629, 416, 10, 236, 4, 10112]
Into a padded sequence: [    3   544  1838    13  3909  3366     4   658  2629   416    10   236
     4 10112     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
is 7
's 8
it 9
in 10
18760


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [20]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [13]:
# tcn_full_summary(model_0)

In [14]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [16]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
192/192 [==============================] - 35s 153ms/step - loss: 0.7005 - accuracy: 0.4913 - val_loss: 0.7378 - val_accuracy: 0.4564
Epoch 2/100
192/192 [==============================] - 27s 142ms/step - loss: 0.6360 - accuracy: 0.6770 - val_loss: 0.5123 - val_accuracy: 0.7629
Epoch 3/100
192/192 [==============================] - 27s 139ms/step - loss: 0.3338 - accuracy: 0.8650 - val_loss: 0.4890 - val_accuracy: 0.7601
Epoch 4/100
192/192 [==============================] - 27s 142ms/step - loss: 0.1534 - accuracy: 0.9514 - val_loss: 0.6659 - val_accuracy: 0.7582
Epoch 5/100
192/192 [==============================] - 33s 171ms/step - loss: 0.0736 - accuracy: 0.9797 - val_loss: 0.8269 - val_accuracy: 0.7591
Epoch 6/100
192/192 [==============================] - 29s 151ms/step - loss: 0.0289 - accuracy: 0.9917 - val_loss: 1.2624 - val_accuracy:

Epoch 10/100
192/192 [==============================] - 32s 167ms/step - loss: 0.0167 - accuracy: 0.9961 - val_loss: 1.8451 - val_accuracy: 0.7233
Epoch 11/100
192/192 [==============================] - 27s 138ms/step - loss: 0.0061 - accuracy: 0.9983 - val_loss: 1.8817 - val_accuracy: 0.7458
Epoch 12/100
192/192 [==============================] - 27s 139ms/step - loss: 0.0036 - accuracy: 0.9990 - val_loss: 2.7956 - val_accuracy: 0.7355
Epoch 13/100
192/192 [==============================] - 27s 139ms/step - loss: 6.1936e-04 - accuracy: 1.0000 - val_loss: 3.3443 - val_accuracy: 0.7467
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 76.54784321784973
Epoch 1/100
192/192 [==============================] - 31s 141ms/step - loss: 0.7078 - accuracy: 0.5136 - val_loss: 0.6332 - val_accuracy: 0.6529
Epoch 2/100
192/192 [==============================] - 30s 158ms/step - loss: 0.5262 - accuracy: 0.7499 - val_loss: 0.4822 - val_accuracy: 0.7570

Epoch 3/100
192/192 [==============================] - 32s 165ms/step - loss: 0.2910 - accuracy: 0.8837 - val_loss: 0.5021 - val_accuracy: 0.7777
Epoch 4/100
192/192 [==============================] - 32s 167ms/step - loss: 0.1163 - accuracy: 0.9618 - val_loss: 0.6769 - val_accuracy: 0.7608
Epoch 5/100
192/192 [==============================] - 33s 171ms/step - loss: 0.0485 - accuracy: 0.9858 - val_loss: 1.1643 - val_accuracy: 0.7561
Epoch 6/100
192/192 [==============================] - 32s 165ms/step - loss: 0.0260 - accuracy: 0.9927 - val_loss: 1.3582 - val_accuracy: 0.7533
Epoch 7/100
192/192 [==============================] - 28s 148ms/step - loss: 0.0213 - accuracy: 0.9928 - val_loss: 1.9645 - val_accuracy: 0.7467
Epoch 8/100
192/192 [==============================] - 28s 147ms/step - loss: 0.0157 - accuracy: 0.9957 - val_loss: 2.0094 - val_accuracy: 0.7552
Epoch 9/100
192/192 [==============================] - 35s 181ms/step - loss: 0.0165 - accuracy: 0.9967 - val_loss: 1.9070 -

Epoch 2/100
192/192 [==============================] - 26s 135ms/step - loss: 0.4660 - accuracy: 0.7842 - val_loss: 0.4690 - val_accuracy: 0.7751
Epoch 3/100
192/192 [==============================] - 26s 134ms/step - loss: 0.1867 - accuracy: 0.9335 - val_loss: 0.5323 - val_accuracy: 0.7751
Epoch 4/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0664 - accuracy: 0.9809 - val_loss: 0.7387 - val_accuracy: 0.7788
Epoch 5/100
192/192 [==============================] - 27s 138ms/step - loss: 0.0333 - accuracy: 0.9895 - val_loss: 1.0956 - val_accuracy: 0.7694
Epoch 6/100
192/192 [==============================] - 26s 137ms/step - loss: 0.0220 - accuracy: 0.9933 - val_loss: 1.5211 - val_accuracy: 0.7507
Epoch 7/100
192/192 [==============================] - 26s 137ms/step - loss: 0.0178 - accuracy: 0.9957 - val_loss: 1.4685 - val_accuracy: 0.7610
Epoch 8/100
192/192 [==============================] - 26s 137ms/step - loss: 0.0196 - accuracy: 0.9936 - val_loss: 1.0915 -

...

Epoch 11/100
192/192 [==============================] - 26s 137ms/step - loss: 0.0098 - accuracy: 0.9968 - val_loss: 1.7037 - val_accuracy: 0.7176
Epoch 12/100
192/192 [==============================] - 26s 136ms/step - loss: 0.0149 - accuracy: 0.9952 - val_loss: 1.5196 - val_accuracy: 0.7139
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 76.54784321784973
Epoch 1/100
192/192 [==============================] - 32s 143ms/step - loss: 0.6592 - accuracy: 0.5893 - val_loss: 0.5544 - val_accuracy: 0.7092
Epoch 2/100
192/192 [==============================] - 27s 139ms/step - loss: 0.3544 - accuracy: 0.8556 - val_loss: 0.5416 - val_accuracy: 0.7608
Epoch 3/100
192/192 [==============================] - 26s 138ms/step - loss: 0.1730 - accuracy: 0.9396 - val_loss: 0.5815 - val_accuracy: 0.7674
Epoch 4/100
192/192 [==============================] - 27s 139ms/step - loss: 0.1028 - accuracy: 0.9669 - val_loss: 0.8428 - val_accuracy: 0.7533
Epoch

Epoch 1/100
192/192 [==============================] - 30s 137ms/step - loss: 0.6851 - accuracy: 0.5551 - val_loss: 0.5318 - val_accuracy: 0.7395
Epoch 2/100
192/192 [==============================] - 26s 134ms/step - loss: 0.3995 - accuracy: 0.8374 - val_loss: 0.5203 - val_accuracy: 0.7535
Epoch 3/100
192/192 [==============================] - 26s 133ms/step - loss: 0.2307 - accuracy: 0.9143 - val_loss: 0.6548 - val_accuracy: 0.7432
Epoch 4/100
192/192 [==============================] - 26s 134ms/step - loss: 0.1256 - accuracy: 0.9597 - val_loss: 0.7062 - val_accuracy: 0.7395
Epoch 5/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0784 - accuracy: 0.9726 - val_loss: 0.9222 - val_accuracy: 0.7441
Epoch 6/100
192/192 [==============================] - 26s 136ms/step - loss: 0.0481 - accuracy: 0.9845 - val_loss: 0.9655 - val_accuracy: 0.7470
Epoch 7/100
192/192 [==============================] - 26s 136ms/step - loss: 0.0380 - accuracy: 0.9889 - val_loss: 1.1138 -

Epoch 3/100
192/192 [==============================] - 26s 134ms/step - loss: 0.2857 - accuracy: 0.8925 - val_loss: 0.5713 - val_accuracy: 0.7392
Epoch 4/100
192/192 [==============================] - 26s 134ms/step - loss: 0.1586 - accuracy: 0.9498 - val_loss: 0.7208 - val_accuracy: 0.7439
Epoch 5/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0864 - accuracy: 0.9746 - val_loss: 0.8853 - val_accuracy: 0.7505
Epoch 6/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0651 - accuracy: 0.9815 - val_loss: 0.9309 - val_accuracy: 0.7373
Epoch 7/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0471 - accuracy: 0.9881 - val_loss: 1.0199 - val_accuracy: 0.7420
Epoch 8/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0413 - accuracy: 0.9876 - val_loss: 1.0766 - val_accuracy: 0.7336
Epoch 9/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0287 - accuracy: 0.9931 - val_loss: 1.1806 -

Epoch 5/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0879 - accuracy: 0.9739 - val_loss: 0.7739 - val_accuracy: 0.7505
Epoch 6/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0577 - accuracy: 0.9856 - val_loss: 0.9955 - val_accuracy: 0.7467
Epoch 7/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0434 - accuracy: 0.9878 - val_loss: 0.9787 - val_accuracy: 0.7533
Epoch 8/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0409 - accuracy: 0.9905 - val_loss: 1.0967 - val_accuracy: 0.7430
Epoch 9/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0296 - accuracy: 0.9924 - val_loss: 1.0520 - val_accuracy: 0.7542
Epoch 10/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0166 - accuracy: 0.9979 - val_loss: 1.1902 - val_accuracy: 0.7514
Epoch 11/100
192/192 [==============================] - 26s 136ms/step - loss: 0.0251 - accuracy: 0.9925 - val_loss: 1.2185

Epoch 12/100
192/192 [==============================] - 26s 133ms/step - loss: 0.0108 - accuracy: 0.9965 - val_loss: 1.6130 - val_accuracy: 0.7317
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 76.92307829856873
Epoch 1/100
192/192 [==============================] - 30s 136ms/step - loss: 0.6716 - accuracy: 0.5664 - val_loss: 0.5371 - val_accuracy: 0.7270
Epoch 2/100
192/192 [==============================] - 25s 131ms/step - loss: 0.3875 - accuracy: 0.8331 - val_loss: 0.5225 - val_accuracy: 0.7430
Epoch 3/100
192/192 [==============================] - 25s 132ms/step - loss: 0.2109 - accuracy: 0.9271 - val_loss: 0.6430 - val_accuracy: 0.7364
Epoch 4/100
192/192 [==============================] - 26s 134ms/step - loss: 0.1030 - accuracy: 0.9660 - val_loss: 0.8623 - val_accuracy: 0.7326
Epoch 5/100
192/192 [==============================] - 26s 133ms/step - loss: 0.0587 - accuracy: 0.9822 - val_loss: 0.9522 - val_accuracy: 0.7383
Epoch 

## Summary

In [17]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
5,relu,6,76.007497,76.382381,77.861166,76.735461,76.829267,76.454031,79.549718,78.611630,76.454031,78.424013,77.330920
0,relu,1,76.569819,77.694470,77.673548,76.547843,76.172608,77.673548,78.142589,77.767354,76.547843,77.485931,77.227556
9,tanh,4,75.913775,76.569819,76.360226,78.236395,75.046903,75.797373,76.547843,76.923078,76.829267,74.859285,76.308396
6,tanh,1,77.788192,75.257730,74.953097,75.140715,77.767354,76.547843,75.984991,76.172608,76.078796,76.360226,76.205155
11,tanh,6,76.944703,74.882847,76.547843,77.954972,75.609756,76.923078,74.390244,75.797373,74.765480,76.172608,75.998890
7,tanh,2,74.039364,76.101220,74.671668,77.016884,78.799248,75.328332,76.641649,75.797373,75.797373,75.422138,75.961525
8,tanh,3,75.726336,78.631675,75.140715,76.923078,73.733586,74.577862,75.609756,75.515950,76.360226,76.360226,75.857941
10,tanh,5,75.351453,75.820059,74.577862,77.954972,75.891185,74.577862,75.609756,75.328332,75.609756,75.891185,75.661242
1,relu,2,50.046861,78.256792,78.236395,78.799248,75.797373,78.799248,77.392119,79.924953,74.671668,78.893059,75.081772
2,relu,3,76.663542,76.382381,76.923078,48.405254,77.861166,79.455912,75.609756,76.641649,78.705442,79.737335,74.638551


In [18]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,77.330920
tanh,76.308396


In [19]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_MR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 16448 words present from 18760 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [16]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [17]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## TCN Model

In [27]:
def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [28]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_145"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_146 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_145 (Embedding)       (None, 100, 300)     300000      input_146[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_145 (SpatialD (None, 100, 300)     0           embedding_145[0][0]              
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_145[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [45]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [47]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.6):
                
                model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=(Xtest, test_y))

                # evaluate the model
                loss, acc = model.evaluate(Xtest, test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.6):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                else:
                    print('Done!')
                    
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
192/192 [==============================] - 19s 76ms/step - loss: 0.7416 - accuracy: 0.4977 - val_loss: 0.6932 - val_accuracy: 0.4845
Epoch 2/100
192/192 [==============================] - 14s 73ms/step - loss: 0.6932 - accuracy: 0.4913 - val_loss: 0.6933 - val_accuracy: 0.4845
Epoch 3/100
192/192 [==============================] - 15s 77ms/step - loss: 0.6932 - accuracy: 0.5026 - val_loss: 0.6933 - val_accuracy: 0.4845
Epoch 4/100
192/192 [==============================] - 17s 86ms/step - loss: 0.6932 - accuracy: 0.4978 - val_loss: 0.6933 - val_accuracy: 0.4845
Epoch 5/100
192/192 [==============================] - 19s 101ms/step - loss: 0.6931 - accuracy: 0.5030 - val_loss: 0.6933 - val_accuracy: 0.4845
Epoch 6/100
192/192 [==============================] - 15s 76ms/step - loss: 0.6932 - accuracy: 0.501

Epoch 6/100
192/192 [==============================] - 19s 100ms/step - loss: 0.3455 - accuracy: 0.8546 - val_loss: 0.4266 - val_accuracy: 0.8116
Epoch 7/100
192/192 [==============================] - 15s 79ms/step - loss: 0.2878 - accuracy: 0.8762 - val_loss: 0.4870 - val_accuracy: 0.7976
Epoch 8/100
192/192 [==============================] - 18s 96ms/step - loss: 0.2500 - accuracy: 0.8988 - val_loss: 0.4877 - val_accuracy: 0.8022
Epoch 9/100
192/192 [==============================] - 15s 76ms/step - loss: 0.2112 - accuracy: 0.9199 - val_loss: 0.5191 - val_accuracy: 0.7863
Epoch 10/100
192/192 [==============================] - 15s 77ms/step - loss: 0.1747 - accuracy: 0.9319 - val_loss: 0.6348 - val_accuracy: 0.7807
Epoch 11/100
192/192 [==============================] - 15s 77ms/step - loss: 0.1583 - accuracy: 0.9431 - val_loss: 0.6153 - val_accuracy: 0.7816
Epoch 12/100
192/192 [==============================] - 15s 78ms/step - loss: 0.1206 - accuracy: 0.9550 - val_loss: 0.7553 - va

192/192 [==============================] - 15s 78ms/step - loss: 0.4263 - accuracy: 0.8005 - val_loss: 0.4654 - val_accuracy: 0.7674
Epoch 4/100
192/192 [==============================] - 15s 76ms/step - loss: 0.3873 - accuracy: 0.8318 - val_loss: 0.4292 - val_accuracy: 0.8021
Epoch 5/100
192/192 [==============================] - 15s 76ms/step - loss: 0.3373 - accuracy: 0.8538 - val_loss: 0.4293 - val_accuracy: 0.7936
Epoch 6/100
192/192 [==============================] - 15s 77ms/step - loss: 0.2733 - accuracy: 0.8907 - val_loss: 0.4351 - val_accuracy: 0.7927
Epoch 7/100
192/192 [==============================] - 14s 75ms/step - loss: 0.2507 - accuracy: 0.8956 - val_loss: 0.5792 - val_accuracy: 0.7814
Epoch 8/100
192/192 [==============================] - 15s 76ms/step - loss: 0.2154 - accuracy: 0.9159 - val_loss: 0.5179 - val_accuracy: 0.7974
Epoch 9/100
192/192 [==============================] - 15s 76ms/step - loss: 0.1799 - accuracy: 0.9314 - val_loss: 0.5610 - val_accuracy: 0.79

Epoch 14/100
192/192 [==============================] - 15s 78ms/step - loss: 0.0748 - accuracy: 0.9717 - val_loss: 0.8298 - val_accuracy: 0.7871
Epoch 15/100
192/192 [==============================] - 15s 79ms/step - loss: 0.0785 - accuracy: 0.9707 - val_loss: 0.8474 - val_accuracy: 0.7842
Epoch 16/100
192/192 [==============================] - 15s 78ms/step - loss: 0.0552 - accuracy: 0.9796 - val_loss: 1.0143 - val_accuracy: 0.7617
Epoch 17/100
192/192 [==============================] - 15s 78ms/step - loss: 0.1009 - accuracy: 0.9633 - val_loss: 0.8842 - val_accuracy: 0.7852
Epoch 18/100
192/192 [==============================] - 15s 78ms/step - loss: 0.0614 - accuracy: 0.9789 - val_loss: 0.9632 - val_accuracy: 0.7852
Epoch 19/100
192/192 [==============================] - 15s 78ms/step - loss: 0.0484 - accuracy: 0.9832 - val_loss: 1.0791 - val_accuracy: 0.7992
Epoch 20/100
192/192 [==============================] - 15s 77ms/step - loss: 0.0358 - accuracy: 0.9876 - val_loss: 0.9167 -

Epoch 1/100
192/192 [==============================] - 21s 86ms/step - loss: 0.7287 - accuracy: 0.5501 - val_loss: 0.5111 - val_accuracy: 0.7432
Epoch 2/100
192/192 [==============================] - 15s 77ms/step - loss: 0.4820 - accuracy: 0.7650 - val_loss: 0.4785 - val_accuracy: 0.7704
Epoch 3/100
192/192 [==============================] - 16s 83ms/step - loss: 0.4400 - accuracy: 0.8017 - val_loss: 0.4909 - val_accuracy: 0.7694
Epoch 4/100
192/192 [==============================] - 20s 104ms/step - loss: 0.4010 - accuracy: 0.8142 - val_loss: 0.4825 - val_accuracy: 0.7676
Epoch 5/100
192/192 [==============================] - 18s 94ms/step - loss: 0.3697 - accuracy: 0.8365 - val_loss: 0.5367 - val_accuracy: 0.7460
Epoch 6/100
192/192 [==============================] - 20s 104ms/step - loss: 0.3096 - accuracy: 0.8616 - val_loss: 0.5044 - val_accuracy: 0.7666
Epoch 7/100
192/192 [==============================] - 15s 76ms/step - loss: 0.2637 - accuracy: 0.8869 - val_loss: 0.6177 - val_

...

Epoch 1/100
192/192 [==============================] - 20s 83ms/step - loss: 0.7293 - accuracy: 0.5573 - val_loss: 0.4872 - val_accuracy: 0.7666
Epoch 2/100
192/192 [==============================] - 16s 81ms/step - loss: 0.4914 - accuracy: 0.7639 - val_loss: 0.4660 - val_accuracy: 0.7713
Epoch 3/100
192/192 [==============================] - 20s 106ms/step - loss: 0.4369 - accuracy: 0.8005 - val_loss: 0.4517 - val_accuracy: 0.7844
Epoch 4/100
192/192 [==============================] - 15s 78ms/step - loss: 0.3776 - accuracy: 0.8334 - val_loss: 0.4793 - val_accuracy: 0.7882
Epoch 5/100
192/192 [==============================] - 15s 76ms/step - loss: 0.3307 - accuracy: 0.8585 - val_loss: 0.5142 - val_accuracy: 0.7470
Epoch 6/100
192/192 [==============================] - 15s 76ms/step - loss: 0.3130 - accuracy: 0.8584 - val_loss: 0.4900 - val_accuracy: 0.7882
Epoch 7/100
192/192 [==============================] - 15s 76ms/step - loss: 0.2472 - accuracy: 0.8942 - val_loss: 0.4924 - val_a

192/192 [==============================] - 15s 78ms/step - loss: 0.0940 - accuracy: 0.9656 - val_loss: 0.9027 - val_accuracy: 0.7598
Epoch 15/100
192/192 [==============================] - 15s 77ms/step - loss: 0.0761 - accuracy: 0.9710 - val_loss: 0.7808 - val_accuracy: 0.7467
Epoch 16/100
192/192 [==============================] - 15s 78ms/step - loss: 0.0720 - accuracy: 0.9759 - val_loss: 1.0275 - val_accuracy: 0.7739
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 79.36210036277771
Done!
Epoch 1/100
192/192 [==============================] - 20s 82ms/step - loss: 0.7171 - accuracy: 0.5116 - val_loss: 0.5098 - val_accuracy: 0.7514
Epoch 2/100
192/192 [==============================] - 15s 77ms/step - loss: 0.5287 - accuracy: 0.7438 - val_loss: 0.5009 - val_accuracy: 0.7533
Epoch 3/100
192/192 [==============================] - 15s 76ms/step - loss: 0.4210 - accuracy: 0.8074 - val_loss: 0.4654 - val_accuracy: 0.7758
Epoch 4/100
192/1

Epoch 10/100
192/192 [==============================] - 15s 76ms/step - loss: 0.1534 - accuracy: 0.9424 - val_loss: 0.5310 - val_accuracy: 0.8041
Epoch 11/100
192/192 [==============================] - 15s 76ms/step - loss: 0.1329 - accuracy: 0.9518 - val_loss: 0.6851 - val_accuracy: 0.7938
Epoch 12/100
192/192 [==============================] - 15s 76ms/step - loss: 0.1187 - accuracy: 0.9583 - val_loss: 0.6036 - val_accuracy: 0.7863
Epoch 13/100
192/192 [==============================] - 15s 78ms/step - loss: 0.1135 - accuracy: 0.9627 - val_loss: 0.6491 - val_accuracy: 0.8051
Epoch 14/100
192/192 [==============================] - 15s 76ms/step - loss: 0.0952 - accuracy: 0.9660 - val_loss: 0.6404 - val_accuracy: 0.7985
Epoch 15/100
192/192 [==============================] - 15s 77ms/step - loss: 0.0848 - accuracy: 0.9711 - val_loss: 0.7253 - val_accuracy: 0.7798
Epoch 16/100
192/192 [==============================] - 15s 77ms/step - loss: 0.0696 - accuracy: 0.9761 - val_loss: 0.8898 -

Epoch 20/100
192/192 [==============================] - 15s 77ms/step - loss: 0.0556 - accuracy: 0.9824 - val_loss: 1.0511 - val_accuracy: 0.7720
Epoch 21/100
192/192 [==============================] - 14s 74ms/step - loss: 0.0441 - accuracy: 0.9838 - val_loss: 0.9092 - val_accuracy: 0.7824
Epoch 22/100
192/192 [==============================] - 15s 76ms/step - loss: 0.0438 - accuracy: 0.9847 - val_loss: 1.0473 - val_accuracy: 0.7739
Epoch 23/100
192/192 [==============================] - 14s 75ms/step - loss: 0.0536 - accuracy: 0.9826 - val_loss: 0.9105 - val_accuracy: 0.7842
Epoch 24/100
192/192 [==============================] - 14s 74ms/step - loss: 0.0482 - accuracy: 0.9840 - val_loss: 0.9587 - val_accuracy: 0.7683
Epoch 25/100
192/192 [==============================] - 15s 76ms/step - loss: 0.0430 - accuracy: 0.9864 - val_loss: 1.2440 - val_accuracy: 0.7899
Epoch 26/100
192/192 [==============================] - 14s 75ms/step - loss: 0.0431 - accuracy: 0.9851 - val_loss: 1.1133 -

192/192 [==============================] - 15s 78ms/step - loss: 0.1117 - accuracy: 0.9600 - val_loss: 0.6223 - val_accuracy: 0.8039
Epoch 13/100
192/192 [==============================] - 15s 77ms/step - loss: 0.1124 - accuracy: 0.9609 - val_loss: 0.6821 - val_accuracy: 0.8002
Epoch 14/100
192/192 [==============================] - 15s 78ms/step - loss: 0.0893 - accuracy: 0.9703 - val_loss: 0.5868 - val_accuracy: 0.7908
Epoch 15/100
192/192 [==============================] - 15s 80ms/step - loss: 0.0919 - accuracy: 0.9678 - val_loss: 0.7016 - val_accuracy: 0.7946
Epoch 16/100
192/192 [==============================] - 16s 83ms/step - loss: 0.0749 - accuracy: 0.9747 - val_loss: 0.7068 - val_accuracy: 0.8002
Epoch 17/100
192/192 [==============================] - 16s 84ms/step - loss: 0.0655 - accuracy: 0.9780 - val_loss: 0.7890 - val_accuracy: 0.7861
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 81.89493417739868
Done!
Epoch 1/100
19

## Summary

In [48]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
4,relu,5,78.819120,83.317715,81.050658,79.268295,79.362100,79.455912,80.675423,80.112571,80.675423,79.924953,80.266217
5,relu,6,81.162137,79.006559,77.954972,81.988746,81.332082,79.549718,79.455912,81.050658,81.894934,78.986865,80.238258
2,relu,3,80.318648,81.443298,78.611630,80.393994,80.675423,80.487806,77.767354,81.332082,80.863041,80.487806,80.238108
3,relu,4,80.412370,80.506092,79.174483,79.455912,80.300188,80.956846,80.018759,79.174483,80.300188,81.425893,80.172521
0,relu,1,79.756325,81.162137,77.579737,80.863041,80.393994,80.206376,79.549718,80.018759,81.425893,79.549718,80.050570
1,relu,2,78.537959,80.037487,81.238276,78.705442,79.455912,79.080677,81.144464,80.956846,80.112571,80.206376,79.947601


In [49]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,80.266217


In [50]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_MR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [21]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [22]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

## Train and Test the Model

In [23]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=6, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [24]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.6):
                
                model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=(Xtest, test_y))

                # evaluate the model
                loss, acc = model.evaluate(Xtest, test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.6):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                else:
                    print('Done!')
                    
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
192/192 [==============================] - 49s 197ms/step - loss: 0.7118 - accuracy: 0.5602 - val_loss: 0.4676 - val_accuracy: 0.7826
Epoch 2/100
192/192 [==============================] - 28s 146ms/step - loss: 0.3818 - accuracy: 0.8371 - val_loss: 0.4562 - val_accuracy: 0.7854
Epoch 3/100
192/192 [==============================] - 28s 147ms/step - loss: 0.2060 - accuracy: 0.9239 - val_loss: 0.5029 - val_accuracy: 0.7994
Epoch 4/100
192/192 [==============================] - 26s 137ms/step - loss: 0.1064 - accuracy: 0.9635 - val_loss: 0.6787 - val_accuracy: 0.7694
Epoch 5/100
192/192 [==============================] - 26s 138ms/step - loss: 0.0562 - accuracy: 0.9827 - val_loss: 0.8457 - val_accuracy: 0.7844
Epoch 6/100
192/192 [==============================] - 27s 142ms/step - loss: 0.0310 - accuracy: 0.9898 - val_loss: 0.9863 - val_accuracy: 0.7648
Epoch 7/100
192/192 [==============================] - 29s 151ms/step - loss: 0.0330 - accuracy: 0.9873 - val_loss: 1.0946 -

Epoch 2/100
192/192 [==============================] - 27s 140ms/step - loss: 0.4514 - accuracy: 0.7864 - val_loss: 0.4662 - val_accuracy: 0.7814
Epoch 3/100
192/192 [==============================] - 28s 144ms/step - loss: 0.2620 - accuracy: 0.9011 - val_loss: 0.4687 - val_accuracy: 0.7936
Epoch 4/100
192/192 [==============================] - 34s 177ms/step - loss: 0.1293 - accuracy: 0.9557 - val_loss: 0.6480 - val_accuracy: 0.7936
Epoch 5/100
192/192 [==============================] - 33s 174ms/step - loss: 0.0681 - accuracy: 0.9767 - val_loss: 0.8265 - val_accuracy: 0.7908
Epoch 6/100
192/192 [==============================] - 31s 161ms/step - loss: 0.0411 - accuracy: 0.9860 - val_loss: 0.9592 - val_accuracy: 0.7720
Epoch 7/100
192/192 [==============================] - 34s 176ms/step - loss: 0.0303 - accuracy: 0.9909 - val_loss: 1.0701 - val_accuracy: 0.7833
Epoch 8/100
192/192 [==============================] - 31s 161ms/step - loss: 0.0236 - accuracy: 0.9920 - val_loss: 1.3309 -

192/192 [==============================] - 27s 139ms/step - loss: 0.0311 - accuracy: 0.9909 - val_loss: 1.0342 - val_accuracy: 0.7751
Epoch 8/100
192/192 [==============================] - 30s 158ms/step - loss: 0.0272 - accuracy: 0.9918 - val_loss: 1.5069 - val_accuracy: 0.7582
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 79.75632548332214
Done!
Epoch 1/100
192/192 [==============================] - 33s 147ms/step - loss: 0.7789 - accuracy: 0.5448 - val_loss: 0.4837 - val_accuracy: 0.7545
Epoch 2/100
192/192 [==============================] - 30s 158ms/step - loss: 0.4183 - accuracy: 0.8092 - val_loss: 0.4321 - val_accuracy: 0.7910
Epoch 3/100
192/192 [==============================] - 32s 168ms/step - loss: 0.2463 - accuracy: 0.9063 - val_loss: 0.5101 - val_accuracy: 0.7779
Epoch 4/100
192/192 [==============================] - 33s 170ms/step - loss: 0.1231 - accuracy: 0.9583 - val_loss: 0.5680 - val_accuracy: 0.7723
Epoch 5/100
1

...

Epoch 5/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0510 - accuracy: 0.9838 - val_loss: 0.7956 - val_accuracy: 0.7645
Epoch 6/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0467 - accuracy: 0.9856 - val_loss: 1.0274 - val_accuracy: 0.7692
Epoch 7/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0290 - accuracy: 0.9895 - val_loss: 1.1886 - val_accuracy: 0.7608
Epoch 8/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0178 - accuracy: 0.9951 - val_loss: 1.0634 - val_accuracy: 0.7570
Epoch 9/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0124 - accuracy: 0.9970 - val_loss: 1.5581 - val_accuracy: 0.7627
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 78.8930594921112
Done!
Epoch 1/100
192/192 [==============================] - 31s 139ms/step - loss: 0.7474 - accuracy: 0.5269 - val_loss: 0.4993 - val_accuracy: 0.7580
Ep

Epoch 2/100
192/192 [==============================] - 26s 133ms/step - loss: 0.6932 - accuracy: 0.4997 - val_loss: 0.6935 - val_accuracy: 0.4737
Epoch 3/100
192/192 [==============================] - 26s 133ms/step - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6933 - val_accuracy: 0.4737
Epoch 4/100
192/192 [==============================] - 26s 135ms/step - loss: 0.6959 - accuracy: 0.5079 - val_loss: 0.6949 - val_accuracy: 0.4737
Epoch 5/100
192/192 [==============================] - 26s 135ms/step - loss: 0.6934 - accuracy: 0.5018 - val_loss: 0.6942 - val_accuracy: 0.4737
Epoch 6/100
192/192 [==============================] - 26s 135ms/step - loss: 0.6935 - accuracy: 0.4946 - val_loss: 0.6939 - val_accuracy: 0.4737
Epoch 7/100
192/192 [==============================] - 26s 134ms/step - loss: 0.6933 - accuracy: 0.4952 - val_loss: 0.6937 - val_accuracy: 0.4737
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 47.37335741519928
The mod

192/192 [==============================] - 25s 132ms/step - loss: 0.6583 - accuracy: 0.5705 - val_loss: 0.4959 - val_accuracy: 0.7795
Epoch 5/100
192/192 [==============================] - 26s 135ms/step - loss: 0.3738 - accuracy: 0.8152 - val_loss: 0.4755 - val_accuracy: 0.7777
Epoch 6/100
192/192 [==============================] - 26s 134ms/step - loss: 0.2217 - accuracy: 0.9026 - val_loss: 0.5486 - val_accuracy: 0.7861
Epoch 7/100
192/192 [==============================] - 26s 134ms/step - loss: 0.1324 - accuracy: 0.9578 - val_loss: 0.5524 - val_accuracy: 0.7786
Epoch 8/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0866 - accuracy: 0.9801 - val_loss: 0.9050 - val_accuracy: 0.7730
Epoch 9/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0632 - accuracy: 0.9849 - val_loss: 1.2605 - val_accuracy: 0.7702
Epoch 10/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0558 - accuracy: 0.9874 - val_loss: 1.6022 - val_accura

Epoch 1/100
192/192 [==============================] - 36s 164ms/step - loss: 0.8863 - accuracy: 0.5508 - val_loss: 0.4268 - val_accuracy: 0.8013
Epoch 2/100
192/192 [==============================] - 26s 133ms/step - loss: 0.4022 - accuracy: 0.8230 - val_loss: 0.4100 - val_accuracy: 0.8116
Epoch 3/100
192/192 [==============================] - 25s 133ms/step - loss: 0.2126 - accuracy: 0.9251 - val_loss: 0.5371 - val_accuracy: 0.8022
Epoch 4/100
192/192 [==============================] - 25s 133ms/step - loss: 0.1049 - accuracy: 0.9638 - val_loss: 0.5591 - val_accuracy: 0.7891
Epoch 5/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0656 - accuracy: 0.9801 - val_loss: 0.7437 - val_accuracy: 0.7826
Epoch 6/100
192/192 [==============================] - 26s 134ms/step - loss: 0.0299 - accuracy: 0.9911 - val_loss: 1.0037 - val_accuracy: 0.7648
Epoch 7/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0291 - accuracy: 0.9909 - val_loss: 1.1773 -

192/192 [==============================] - 31s 138ms/step - loss: 0.7597 - accuracy: 0.4951 - val_loss: 0.6927 - val_accuracy: 0.5582
Epoch 2/100
192/192 [==============================] - 26s 134ms/step - loss: 0.6759 - accuracy: 0.5517 - val_loss: 0.5009 - val_accuracy: 0.7636
Epoch 3/100
192/192 [==============================] - 26s 134ms/step - loss: 0.3986 - accuracy: 0.8360 - val_loss: 0.4109 - val_accuracy: 0.8133
Epoch 4/100
192/192 [==============================] - 26s 134ms/step - loss: 0.1939 - accuracy: 0.9292 - val_loss: 0.5367 - val_accuracy: 0.7955
Epoch 5/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0929 - accuracy: 0.9678 - val_loss: 0.6926 - val_accuracy: 0.7777
Epoch 6/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0533 - accuracy: 0.9834 - val_loss: 0.9234 - val_accuracy: 0.7664
Epoch 7/100
192/192 [==============================] - 26s 135ms/step - loss: 0.0362 - accuracy: 0.9888 - val_loss: 0.8256 - val_accurac

## Summary

In [25]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
5,relu,6,81.162137,79.850048,82.363975,80.581611,79.080677,78.517824,81.332082,78.142589,80.018759,78.799248,79.984895
4,relu,5,79.943770,79.662603,75.797373,81.144464,80.206376,78.330207,78.611630,80.863041,81.144464,78.893059,79.459699
2,relu,3,79.662603,79.100281,78.893059,79.643530,77.204502,79.643530,80.769229,79.549718,78.611630,78.611630,79.168971
3,relu,4,78.350514,81.255859,78.048778,79.831147,77.204502,79.737335,81.707317,78.893059,76.172608,80.393994,79.159511
0,relu,1,79.943770,80.599815,78.705442,79.455912,78.611630,79.362100,79.362100,78.893059,78.048778,78.424013,79.140662
1,relu,2,79.756325,79.100281,77.392119,78.142589,79.549718,78.517824,77.861166,81.332082,79.455912,79.080677,79.018869


In [26]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_MR_3.xlsx', sheet_name='dynamic')